In [1]:
import os
from yt_dlp import YoutubeDL
import cv2 
import matplotlib.pyplot as plt

In [2]:
!pip install yt-dlp


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
url = "https://www.youtube.com/watch?v=3D-e1dhUYUw"
ydl_opts = {
    "format": "bestvideo",
    "outtmpl": "video_only.%(ext)s"
}

In [4]:
with YoutubeDL(ydl_opts) as ydl:
    ydl.download([url])

[youtube] Extracting URL: https://www.youtube.com/watch?v=3D-e1dhUYUw
[youtube] 3D-e1dhUYUw: Downloading webpage
[youtube] 3D-e1dhUYUw: Downloading android sdkless player API JSON
[youtube] 3D-e1dhUYUw: Downloading tv client config
[youtube] 3D-e1dhUYUw: Downloading player 87644c66-main
[youtube] 3D-e1dhUYUw: Downloading tv player API JSON
[youtube] 3D-e1dhUYUw: Downloading web safari player API JSON


         player = https://www.youtube.com/s/player/87644c66/player_ias.vflset/en_US/base.js
         n = roTJDBHFzXVnMFh7J71 ; player = https://www.youtube.com/s/player/87644c66/player_ias.vflset/en_US/base.js
         Please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U


[youtube] 3D-e1dhUYUw: Downloading m3u8 information
[info] 3D-e1dhUYUw: Downloading 1 format(s): 299
[download] video_only.mp4 has already been downloaded
[download] 100% of   69.30MiB


In [5]:

# -------- VIDEO SELECT --------
video_path = "video_only.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("❌ Cannot open video")
    exit()

# -------- READ FIRST FRAME --------
ret, frame = cap.read()
if not ret:
    print("❌ Cannot read first frame")
    exit()

# -------- SELECT ROI --------
bbox = cv2.selectROI("Select Object", frame, showCrosshair=True)
cv2.destroyWindow("Select Object")

# -------- TRACKERS --------
trackers = {
    "KCF": cv2.legacy.TrackerKCF_create(),
    "CSRT": cv2.legacy.TrackerCSRT_create()
}

output_dir = "results"
os.makedirs(output_dir, exist_ok=True)

# -------- RUN BOTH TRACKERS --------
for tracker_name, tracker in trackers.items():

    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # reset video
    tracker.init(frame, bbox)
    frame_id = 0

    print(f"\n🚀 Running {tracker_name} tracker...")

    while True:
        ret, frame = cap.read()
        if not ret or frame_id > 15:  # ~15 frames
            break
        
        success, box = tracker.update(frame)

        if success:
            x, y, w, h = map(int, box)
            cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)
            cv2.putText(frame, tracker_name, (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (0,255,0), 2)
        else:
            cv2.putText(frame, "Tracking Lost", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 
                        1, (0,0,255), 2)

        # Save frame
        cv2.imwrite(f"{output_dir}/{tracker_name}_frame_{frame_id}.png", frame)
        frame_id += 1

cap.release()
print("\n✅ Done. Check 'results' folder.")


🚀 Running KCF tracker...

🚀 Running CSRT tracker...

✅ Done. Check 'results' folder.


In [6]:
import subprocess

In [7]:
input_video = "video_only.mp4"
output_video = "clip.mp4"
start_time = "00:03:00"   # начало (3 мин)
duration = "00:00:15"     # длительность (15 сек)

command = [
    "ffmpeg", "-ss", start_time, "-i", input_video,
    "-t", duration, "-c", "copy", output_video
]

subprocess.run(command)
print("✅ Clip saved:", output_video)

FileNotFoundError: [WinError 2] Не удается найти указанный файл